In [1]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_project = os.getenv("LANGCHAIN_PROJECT")
langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")

print(f"OpenAI API: {openai_api_key[:4]}********")
print(f"LangChain API Key: {langchain_api_key[:4]}********")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["LANGCHAIN_PROJECT"] = langchain_project
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = langchain_tracing


OpenAI API: sk-p********
LangChain API Key: lsv2********


LangSmith Setup (LLM Logging)


In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-99d2ee79-537c-478d-a7cc-33a440818509-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

Starts from now

In [4]:
import os
import json
import getpass

# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [5]:
#cd /content/drive/MyDrive/24fall/tenten/tenten-ai

In [6]:
def ms_to_minutes_seconds_str(milliseconds):
    seconds = milliseconds // 1000  # 총 초
    minutes = seconds // 60  # 분 계산
    remaining_seconds = seconds % 60  # 남은 초 계산
    return f"{minutes}분 {remaining_seconds}초"  # 출력 형식

# 파일 
with open('transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = data['utterances']

# 발화를 Document 객체로 생성
documents = []
for utterance in utterances:
    content = utterance['msg']
    metadata = {
        'start_at': utterance['start_at'],  # 밀리초 단위
        'duration': utterance['duration'],   # 밀리초 단위
        'speaker': utterance['spk'],
        'speaker_type': utterance['spk_type']
    }
    documents.append(Document(page_content=content, metadata=metadata))

# 시작 시간 기준으로 정렬
documents = sorted(documents, key=lambda doc: doc.metadata['start_at'])

In [7]:
full_text = ''
for doc in documents:
    start_time_str = ms_to_minutes_seconds_str(doc.metadata['start_at'])
    full_text += f"[{start_time_str}] {doc.page_content}\n"

# 언어 모델 초기화 # mini 버전 써보기
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

C:\Users\yooji\AppData\Local\Temp\ipykernel_14816\1390165639.py:7: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


In [8]:
import re
# 역할 지정해서 돌려보기
#role description
#확정은 결과물 가지고 펠로우님이랑 이야기
#chunk 늘려보기..
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
아래는 상담 세션의 전체 스크립트입니다. 이 스크립트를 시간의 흐름에 따라 **주제별**로 요약해 주세요.

다음 조건을 따라 주세요:
1. 주제는 각 시간 구간에서 **한 가지 주요 질문** 또는 **하나의 조언**으로 요약해 주세요. 여러 주제가 섞이지 않도록 주의하세요.
2. 각 주제에 대해 시작 시간과 종료 시간을 포함해 주세요. 시간은 해당 주제가 처음 등장한 스크립트의 시작 시간과 마지막으로 등장한 스크립트의 시작 시간을 사용하여 계산됩니다.
3. 각 시간 구간에 해당하는 요약은 아래 중 **하나**입니다:
   - 질문인 경우: **Q.** 형식으로 질문을 요약해 주세요.
   - 행동을 변경해야 하는 경우: **[바꾸세요]** 형식으로 요약해 주세요.
   - 권장 사항인 경우: **[이렇게 하세요]** 형식으로 요약해 주세요.

예시:
- 0분 42초 ~ 3분 15초
  - Q. 당뇨 약을 일주일 간 끊어도 될까요?

- 5분 0초 ~ 7분 30초
  - [바꾸세요] 당뇨 약은 중단하지 마세요.

- 10분 15초 ~ 12분 45초
  - [이렇게 하세요] 약 복용을 지속하세요.

스크립트:
{text}

요약:
"""
)
# $0.17
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

# 전체 스크립트 요약 
session_summary = summary_chain.run(text=full_text)
print(session_summary)

# 출력 파싱, 주제와 시간 정보 추출
subtopics = re.findall(r'- (\d+분 \d+초) ~ (\d+분 \d+초)\n\s*- (.+?)(?=\n- \d+분|\Z)', session_summary, re.DOTALL)

# 임베딩 및 벡터스토어 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

C:\Users\yooji\AppData\Local\Temp\ipykernel_14816\158058998.py:33: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  summary_chain = LLMChain(llm=llm, prompt=summary_prompt)
C:\Users\yooji\AppData\Local\Temp\ipykernel_14816\158058998.py:36: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  session_summary = summary_chain.run(text=full_text)


- 0분 42초 ~ 3분 15초
  - Q. 당뇨 약을 일주일 간 끊어도 될까요?

- 3분 37초 ~ 5분 22초
  - [이렇게 하세요] 스테로이드 사용 시 당뇨 관리에 주의하세요.

- 5분 24초 ~ 6분 20초
  - [이렇게 하세요] 무게 없이 운동을 시작하세요.

- 7분 32초 ~ 9분 16초
  - Q. 비염에 아연과 프로폴리스가 효과가 있나요?

- 10분 24초 ~ 12분 20초
  - [이렇게 하세요] 비타민 B를 강화하여 피로 회복을 도와주세요.

- 13분 2초 ~ 15분 20초
  - [이렇게 하세요] 오메가 3를 섭취하여 눈 건강을 개선하세요.

- 16분 6초 ~ 19분 28초
  - [이렇게 하세요] 하체 운동을 통해 다리 부종을 줄이세요.

- 21분 1초 ~ 23분 58초
  - Q. 영양제를 식전과 식후에 어떻게 복용해야 하나요?

- 25분 28초 ~ 26분 32초
  - [이렇게 하세요] 30대 후반부터는 오메가 3와 종합비타민을 섭취하세요.

- 27분 25초 ~ 30분 17초
  - Q. 유산균과 포스트바이오틱스를 함께 먹어야 하나요?

- 31분 15초 ~ 34분 0초
  - Q. 비타민 D와 다른 영양제를 함께 먹어도 되나요?

- 35분 43초 ~ 37분 0초
  - Q. 오메가 3가 건강에 유익한가요?

- 40분 6초 ~ 41분 23초
  - [이렇게 하세요] 비타민 D는 결핍 여부에 따라 복용량을 조절하세요.

- 42분 6초 ~ 44분 56초
  - [이렇게 하세요] 비타민 C를 피로 회복에 활용하세요.

- 50분 25초 ~ 52분 3초
  - Q. 간염 보균자가 복용해야 할 약의 부작용은 무엇인가요?

- 56분 5초 ~ 57분 30초
  - [이렇게 하세요] 간염 약은 24시간 간격으로 복용하세요.

- 60분 12초 ~ 61분 9초
  - [이렇게 하세요] 항산화 영양제를 꾸준히 섭취하세요.

- 62분 22초 ~ 63분 15초
  - Q. 한약이 간에 영향을 미치나요?

- 66분 8초 ~ 67

C:\Users\yooji\AppData\Local\Temp\ipykernel_14816\158058998.py:43: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings()


In [9]:
# 리트리버 
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [10]:
# 각 주제에 대해 retrieve
for idx, (start_time_str, end_time_str, content) in enumerate(subtopics):
    print(f"\n### 주제 {idx + 1}: {start_time_str} ~ {end_time_str} ###\n")
    subtopic = content.strip()
    print(subtopic)

    def time_str_to_ms(time_str):
        match = re.match(r'(\d+)분 (\d+)초', time_str)
        if match:
            minutes = int(match.group(1))
            seconds = int(match.group(2))
            return (minutes * 60 + seconds) * 1000
        else:
            return 0

    start_time_ms = time_str_to_ms(start_time_str)
    end_time_ms = time_str_to_ms(end_time_str)
    
    # 해당 시간에서 retrieve
    retrieved_docs = retriever.get_relevant_documents(subtopic)
   
    relevant_docs = [
        doc for doc in retrieved_docs
        if start_time_ms <= doc.metadata['start_at'] <= end_time_ms
    ]
    
    # 시작 시간 기준으로 정렬
    sorted_relevant_docs = sorted(relevant_docs, key=lambda doc: doc.metadata['start_at'])
    
    # 출력
    start_at_and_context = "\n\n".join(
        [f"{ms_to_minutes_seconds_str(doc.metadata['start_at'])}: {doc.page_content}" for doc in sorted_relevant_docs]
    )
    
    print(f"\n**관련 스크립트 부분:**\n{start_at_and_context}")
    print("\n---\n")


### 주제 1: 0분 42초 ~ 3분 15초 ###

Q. 당뇨 약을 일주일 간 끊어도 될까요?


C:\Users\yooji\AppData\Local\Temp\ipykernel_14816\928118081.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  retrieved_docs = retriever.get_relevant_documents(subtopic)



**관련 스크립트 부분:**
0분 42초: 다른 글 먹더라도, 그리고 예를 들어서 건강검진 받을 때 한 일주일 전에서부터 아스트림 같은 걸 제가 끊어버리거든요.

---


### 주제 2: 3분 37초 ~ 5분 22초 ###

[이렇게 하세요] 스테로이드 사용 시 당뇨 관리에 주의하세요.

**관련 스크립트 부분:**
3분 37초: 근데 스테로이드를 안 써도 이 뼈만 여기에 맞는 주사는 다 좀 올린다고 그러더라고요.

4분 29초: 그래서 치료기관을 단축하기 위해서 스테루제를 저는 짧게 쓰는 것도.

4분 41초: 이게 스테드.

4분 51초: 그래서 이게 통증이 정말 참을 수 없다 하면 스테로드제를 받고 이제 내분비내과 그 당뇨 관리하시는 분한테는 스테로즈 맞았다는 걸 좀 미리 알리고.

---


### 주제 3: 5분 24초 ~ 6분 20초 ###

[이렇게 하세요] 무게 없이 운동을 시작하세요.

**관련 스크립트 부분:**
6분 16초: 여기서 운동을 하지 네, 무게.

6분 18초: 안 들고 하시면 돼요, 무게 없이.

---


### 주제 4: 7분 32초 ~ 9분 16초 ###

Q. 비염에 아연과 프로폴리스가 효과가 있나요?

**관련 스크립트 부분:**
7분 36초: 그래서 제가 비염, 그 프로폴릿 이거 아연 비염이 좋다고 해서 못 봤는데 효과가 있는지 모르겠어요.

8분 27초: 또 프로폴리스도 항염 작용이 많아서 이제 특히 기관지 면역 쪽이 좋거든요. 그래서 지금 보여주신 게 회사가 닥터 아, 둘.

---


### 주제 5: 10분 24초 ~ 12분 20초 ###

[이렇게 하세요] 비타민 B를 강화하여 피로 회복을 도와주세요.

**관련 스크립트 부분:**


---


### 주제 6: 13분 2초 ~ 15분 20초 ###

[이렇게 하세요] 오메가 3를 섭취하여 눈 건강을 개선하세요.

**관련 스크립트 부분:**
14분 59초: 그래서 오히려 오메가 3 가 많이 부족해서 오메가 3 가 근데 눈에 그 수분층 위에 기름층을 형